In [6]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parse XML file
tree = ET.parse('/Users/sudz4/Desktop/SUDZ4DEV/cyber-research/nmap_output/hw3_nmap_findings_4.2.xml')
root = tree.getroot()

# Initialize a list to hold all the scan data
scan_data = []

# Iterate through the hosts in the XML file
for host in root.iter('host'):
    # Get the IP address of the host
    address = host.find('address').get('addr')
    # Iterate through the host's ports
    for port in host.iter('port'):
        # Get the portid and protocol
        portid = port.get('portid')
        protocol = port.get('protocol')
        # Get the service name
        service = port.find('service').get('name')
        # Initialize a list to hold the CVEs for this port
        cves = []
        # Iterate through the scripts
        for script in port.iter('script'):
            # If the script is 'vulners', iterate through the 'table' elements
            if script.get('id') == 'vulners':
                for table in script.iter('table'):
                    # Iterate through the 'elem' elements and add the CVEs to the list
                    for elem in table.iter('elem'):
                        if elem.get('key') == 'ID':
                            cves.append(elem.text)
        # Add the data for this port to the scan data
        scan_data.append({'IP': address, 
                          'Protocol': protocol, 
                          'Port': portid, 
                          'Service': service, 
                          'CVEs': ', '.join(cves)})

# Create a DataFrame from the list of scan data
df = pd.DataFrame(scan_data)

# Print out the DataFrame
print(df)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sudz4/Desktop/SUDZ4DEV/cyber-research/nmap_output/hw3_nmap_findings_4.1.xml'